In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('airbnb.csv')

In [3]:
df

,id,price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,host_identity_verified,host_response_rate,host_since,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds
0,12513361,555.68,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",""Sm...",2,1,Real Bed,flexible,False,NYC,t,89%,18/11/2015,East Harlem,3,87.0,10029,0,1
1,7196412,366.36,Cabin,Private room,"{""Wireless Internet"",Kitchen,Washer,Dryer,""Smo...",2,3,Real Bed,moderate,False,LA,f,100%,10/9/2016,Valley Glen,14,91.0,91606,1,1
2,16333776,482.83,House,Private room,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",2,1,Real Bed,strict,True,SF,t,100%,26/12/2013,Richmond District,117,96.0,94118,1,1
3,1786412,448.86,Apartment,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",2,1,Real Bed,strict,True,NYC,t,93%,11/5/2010,Williamsburg,8,86.0,11211,1,1
4,14575777,506.89,Villa,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",6,2,Real Bed,strict,True,LA,t,70%,22/10/2015,NaN,2,100.0,90703,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,20257960,478.75,House,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1,Real Bed,flexible,False,LA,t,NaN,6/3/2013,Mid-Wilshire,0,NaN,90035,1,1
156,4559312,510.59,Apartment,Entire home/apt,"{Internet,""Wireless Internet"",""Air conditionin...",4,1,Real Bed,strict,False,NYC,t,NaN,6/3/2013,Harlem,2,80.0,10026,2,2
157,2165604,368.89,Apartment,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",2,1,Real Bed,moderate,True,NYC,t,100%,14/3/2016,Harlem,7,94.0,10039,1,1
158,18385413,409.43,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",2,1,Real Bed,flexible,False,NYC,f,NaN,4/6/2014,Harlem,18,91.0,10030,0,1


In [ ]:
#1 Find the count of verified and non-verified Airbnb hosts

select host_identity_verified,
count(*)
from airbnb_search_details 
group by host_identity_verified ;

In [ ]:
#2 List neighbourhood with parking space with no cleaning fees

select neighbourhood,amenities,cleaning_fee 
from airbnb_search_details 
where amenities LIKE '%Free parking on premises%'
and cleaning_fee = 'False';

**Growth of Airbnb**

Estimate the growth of Airbnb each year using the number of hosts registered as the growth metric. 
The rate of growth is calculated by taking ((number of hosts registered in the current year - number of hosts registered in the previous year) / the number of hosts registered in the previous year) * 100.
Output the year, number of hosts in the current year, number of hosts in the previous year, and the rate of growth. 
Round the rate of growth to the nearest percent and order the result in the ascending order based on the year.
Assume that the dataset consists only of unique hosts, meaning there are no duplicate hosts listed.

In [ ]:
#3 Growth of AirBnB

WITH register AS (
SELECT EXTRACT(year FROM host_since) AS year,
COUNT(id) AS num_host
FROM airbnb_search_details
GROUP BY 1)

SELECT 

curr_year.year,
curr_year.num_host AS num_hosts_curr,
prev_year.num_host AS num_host_prev,
ROUND((curr_year.num_host-prev_year.num_host*1.00)/prev_year.num_host*100) AS rate_growth

FROM register curr_year

LEFT JOIN register prev_year
    ON prev_year.year=curr_year.year -1
    
ORDER BY 1

In [ ]:
#4 Find neighborhoods where you can sleep on a real bed in a villa with internet while paying the lowest price possible.

SELECT
    `neighbourhood`,
    `bed_type`,
    `property_type`,
    `amenities`,
    MIN(`price`) AS lowest_price
    
FROM
    air_bnb
    
WHERE
	`amenities` LIKE '%Wireless Internet%' AND `property_type` = 'Villa' AND `bed_type`='Real Bed'
    
GROUP BY `neighbourhood`

categorize the number of reviews into the following groups along with the price.
   
0 reviews: NO

1 to 5 reviews: FEW

6 to 15 reviews: SOME

16 to 40 reviews: MANY

more than 40 reviews: A LOT

In [ ]:
#5 categorize the number of reviews into the following groups along with the price.

SELECT `number_of_reviews`,`price`,
CASE 
WHEN `number_of_reviews` <= 5 THEN 'NO'
WHEN `number_of_reviews` <= 15 THEN 'FEW'
WHEN `number_of_reviews` <= 40 THEN 'MANY'
WHEN `number_of_reviews` < 41 THEN 'A LOT'
END AS review_category

FROM air_bnb

In [ ]:
#6 Find the average number of beds in each neighborhood that has at least 3 beds in total // Output results along with the neighborhood name and sort the results based on the number of average beds in descending order

SELECT `neighbourhood`,AVG(`beds`) as avg_beds
FROM air_bnb
WHERE `beds`>= 3
GROUP BY `neighbourhood`
ORDER BY `avg_beds` DESC

In [ ]:
#7 Find the price of the cheapest property for every city.

SELECT `neighbourhood`,MIN(`price`) AS min_price
FROM air_bnb
GROUP BY `neighbourhood`

In [ ]:
#8 Find the average number of bathrooms and bedrooms for each city’s property types. Output the result along with the city name and the property type.

SELECT `neighbourhood`,`property_type`,ROUND(AVG(`bathrooms`)) AS avg_bath, ROUND(AVG(`bedrooms`)) AS avg_bath
FROM air_bnb
GROUP BY `neighbourhood`

In [ ]:
#9 Table consists of search results and outputs host information for searchers. Find the minimum, average, maximum rental prices for each host’s popularity rating. The host’s popularity rating is defined as below

SELECT
MIN(`price`) AS min_price, MAX(`price`) AS max_price, AVG(`price`) AS avg_price, `number_of_reviews`,
CASE
WHEN `number_of_reviews` = 0 THEN 'New'
WHEN `number_of_reviews` <= 5 THEN 'Rising'
WHEN `number_of_reviews` <= 15 THEN 'Trending Up'
WHEN `number_of_reviews` <= 40 THEN 'Popular'
WHEN `number_of_reviews` > 40 THEN 'Hot'
END AS popularity_rating
   
FROM air_bnb
GROUP BY popularity_rating

In [ ]:
#10 Find all search details where data is missing from the host_response_rate column.

SELECT
    *
FROM
    air_bnb
WHERE `host_response_rate` IS NULL

In [ ]:
#11 Find the search details for villas and houses with wireless internet access.

SELECT *
FROM air_bnb
WHERE `property_type`='Villa' 
AND `amenities` LIKE '%Wireless Internet%'

GROUP BY `neighbourhood`;

In [ ]:
#12 Find searches for Los Angeles neighborhoods.

SELECT *
FROM air_bnb
WHERE `neighbourhood` IN ('Hollywood Hills','Long Beach','Hollywood')

In [ ]:
#13 Find all searches for accommodations where the number of bedrooms is equal to the number of bathrooms.

SELECT `accommodates`,`bedrooms`,`bathrooms`
FROM air_bnb
WHERE `bedrooms`=`bathrooms`
GROUP BY `accommodates`
ORDER BY `accommodates`

In [ ]:
#14 Find the search details of 50 apartment searches the Harlem neighborhood of New York City.

SELECT *
FROM air_bnb
WHERE `property_type` = 'Apartment' AND `neighbourhood` = 'East Harlem'